In [15]:
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image

In [10]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder('aug_processed_data', transform=transform)

# Train/val split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
num_classes = 2

In [11]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 32 * 32, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))   # 64x64
        x = self.pool(F.relu(self.conv2(x)))   # 32x32
        x = x.view(-1, 32 * 32 * 32)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [12]:
## Approch 2


class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)

        self.dropout = nn.Dropout(p=0.5)  # Dropout with 50% probability

        self.fc1 = nn.Linear(32 * 32 * 32, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))   # 128->64
        x = self.pool(F.relu(self.bn2(self.conv2(x))))   # 64->32

        x = x.view(-1, 32 * 32 * 32)
        x = self.dropout(F.relu(self.fc1(x)))            # Dropout before fc2
        x = self.fc2(x)
        return x

In [13]:
model = SimpleCNN()

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # weight_decay added

In [14]:
def evaluate(model, data_loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(data_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy


def train_model(epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            print(f"Batch loss: {loss.item()}")
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = total_loss / len(train_loader)
        train_acc = 100 * correct / total

        val_loss, val_acc = evaluate(model, val_loader, criterion)

        print(f"Epoch {epoch+1} | "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

# Assuming you already have a val_loader for your validation dataset

train_model(epochs=100)

Batch loss: 0.8601464033126831
Batch loss: 18.905698776245117
Batch loss: 2.4714510440826416
Batch loss: 4.326797008514404
Batch loss: 4.848811149597168
Epoch 1 | Train Loss: 6.2826 | Train Acc: 45.00% | Val Loss: 0.6796 | Val Acc: 55.00%
Batch loss: 3.625509738922119
Batch loss: 1.2845743894577026
Batch loss: 1.8352090120315552
Batch loss: 1.3428250551223755
Batch loss: 1.290202021598816
Epoch 2 | Train Loss: 1.8757 | Train Acc: 61.25% | Val Loss: 0.4080 | Val Acc: 75.00%
Batch loss: 0.4759811758995056
Batch loss: 1.0756744146347046
Batch loss: 1.5735763311386108
Batch loss: 1.4803065061569214
Batch loss: 1.1951477527618408
Epoch 3 | Train Loss: 1.1601 | Train Acc: 58.75% | Val Loss: 0.3801 | Val Acc: 72.50%
Batch loss: 0.7290656566619873
Batch loss: 0.4212764799594879
Batch loss: 0.5841568112373352
Batch loss: 0.4425114393234253
Batch loss: 0.3505750000476837
Epoch 4 | Train Loss: 0.5055 | Train Acc: 62.50% | Val Loss: 0.4386 | Val Acc: 45.00%
Batch loss: 0.48222529888153076
Batch lo

Model Testing

In [20]:

def predict_single_image(image_path, model, class_names):
    model.eval()
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    img = Image.open(image_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        output = model(img_tensor)
        probs = F.softmax(output, dim=1)
        _, predicted = torch.max(probs, 1)

    print(f"Predicted Class: {class_names[predicted.item()]}")
    print(f"Class Probabilities: {probs.squeeze().numpy()}")


In [17]:
# Assuming dataset = ImageFolder(...)
class_names = dataset.classes  # ['healthy', 'infected']

# Path to one test image
test_image_path_1 = "processed_data/serie healthy leaves/healthy_04.png"

predict_single_image(test_image_path_1, model, class_names)

Predicted Class: serie_healthy_leaves_augmented
Class Probabilities: [9.991443e-01 8.556470e-04]


In [19]:
test_image_path_2 = "processed_data/serie infected leaves/infected_05.png"
predict_single_image(test_image_path_2, model, class_names)


Predicted Class: series_infected_leaves_augmented
Class Probabilities: [1.7830813e-29 1.0000000e+00]
